In [1]:
import simpy


In [2]:
# Parameters for scaled simulation
NUM_CONTAINERS = 106
NUM_TRUCKS = 9  
NUM_CRANES = 3   

TRUCK_ROUND_TRIP_TIME = 2.0   # hours per container (loading + travel + unloading + return)
CRANE_HANDLING_TIME = 0.5     # hours per container

In [3]:
class Stats:
    def __init__(self):
        self.truck_wait_times = []
        self.crane_wait_times = []
        self.truck_busy_time = 0.0
        self.crane_busy_time = 0.0

In [4]:
def container(env, name, trucks, cranes, stats):
    arrival_time = env.now
    # Request truck and measure wait time
    with trucks.request() as req_truck:
        truck_req_time = env.now
        yield req_truck
        truck_wait = env.now - truck_req_time
        stats.truck_wait_times.append(truck_wait)
        
        print(f"{env.now:.2f}h - {name} loading onto truck after waiting {truck_wait:.2f}h")
        start = env.now
        yield env.timeout(TRUCK_ROUND_TRIP_TIME)
        stats.truck_busy_time += TRUCK_ROUND_TRIP_TIME
        print(f"{env.now:.2f}h - {name} arrived at terminal")


    # Request crane and measure wait time
    with cranes.request() as req_crane:
        crane_req_time = env.now
        yield req_crane
        crane_wait = env.now - crane_req_time
        stats.crane_wait_times.append(crane_wait)
        
        print(f"{env.now:.2f}h - {name} loading onto train after waiting {crane_wait:.2f}h")
        yield env.timeout(CRANE_HANDLING_TIME)
        stats.crane_busy_time += CRANE_HANDLING_TIME
        print(f"{env.now:.2f}h - {name} loaded onto train")

In [5]:
def run_scaled_simulation():
    env = simpy.Environment()
    trucks = simpy.Resource(env, capacity=NUM_TRUCKS)
    cranes = simpy.Resource(env, capacity=NUM_CRANES)
    stats = Stats()

    for i in range(NUM_CONTAINERS):
        env.process(container(env, f"C{i+1}", trucks, cranes, stats))

    env.run(until=24)

    # Calculate and print summary statistics
    avg_truck_wait = sum(stats.truck_wait_times) / len(stats.truck_wait_times)
    avg_crane_wait = sum(stats.crane_wait_times) / len(stats.crane_wait_times)

    truck_util = stats.truck_busy_time / (NUM_TRUCKS * 24) * 100
    crane_util = stats.crane_busy_time / (NUM_CRANES * 24) * 100

    print("\n--- Simulation Summary ---")
    print(f"Average Truck Wait Time: {avg_truck_wait:.2f} hours")
    print(f"Average Crane Wait Time: {avg_crane_wait:.2f} hours")
    print(f"Truck Utilization: {truck_util:.2f}%")
    print(f"Crane Utilization: {crane_util:.2f}%")

if __name__ == "__main__":
    run_scaled_simulation()

0.00h - C1 loading onto truck after waiting 0.00h
0.00h - C2 loading onto truck after waiting 0.00h
0.00h - C3 loading onto truck after waiting 0.00h
0.00h - C4 loading onto truck after waiting 0.00h
0.00h - C5 loading onto truck after waiting 0.00h
0.00h - C6 loading onto truck after waiting 0.00h
0.00h - C7 loading onto truck after waiting 0.00h
0.00h - C8 loading onto truck after waiting 0.00h
0.00h - C9 loading onto truck after waiting 0.00h
2.00h - C1 arrived at terminal
2.00h - C2 arrived at terminal
2.00h - C3 arrived at terminal
2.00h - C4 arrived at terminal
2.00h - C5 arrived at terminal
2.00h - C6 arrived at terminal
2.00h - C7 arrived at terminal
2.00h - C8 arrived at terminal
2.00h - C9 arrived at terminal
2.00h - C1 loading onto train after waiting 0.00h
2.00h - C2 loading onto train after waiting 0.00h
2.00h - C3 loading onto train after waiting 0.00h
2.00h - C10 loading onto truck after waiting 2.00h
2.00h - C11 loading onto truck after waiting 2.00h
2.00h - C12 loading